# Working with vegetation health data

**NOTE**: Currently only one VH product (FAPAR / LAI).


Other Vegetation Health Index Product

from [NOAA STAR - Global Vegetation Health Products](https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/vh_ftp.php)

```
wget --recursive --no-parent ftp://ftp.star.nesdis.noaa.gov/pub/corp/scsb/wguo/data/Blended_VH_4km/VH/
```

[User Guide](https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH_doc/VHP_uguide_v2.0_2018_0727.pdf)

In [1]:
import geopandas as gpd
from pathlib import Path
import xarray as xr
import numpy as np
import pandas as pd

import cdsapi
import matplotlib.pyplot as plt
import cartopy.crs as crs
import xarray as xr
import seaborn as sns

import os
import subprocess
from functools import partial
from collections import namedtuple
from pprint import pprint

%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 6

In [2]:
# Set working directory as project root
if os.getcwd().split('/')[-1] != "ESoWC_dummy":
    os.chdir('..')
    
assert os.getcwd().split('/')[-1] == "ESoWC_dummy", f"Working directory should be the root (), currently: {os.getcwd()}"
print("Current Directory: ",os.getcwd())

Current Directory:  /Users/tommylees/github/ESoWC_dummy


In [3]:
# Set data_dir to external if on tommy's laptop (no space!)
if os.getcwd().split('/')[2] == 'tommylees':
    data_dir = Path("/Volumes/Lees_Extend/data/ecmwf_sowc/")
else:
    data_dir = Path('./data')
    
print(data_dir)
pprint([f.as_posix().split('/')[-1] for f in data_dir.glob('*')])

/Volumes/Lees_Extend/data/ecmwf_sowc
['chirps_kenya.nc',
 'chirps_kenya2.nc',
 'countries_shp',
 'countries_shp.zip',
 'era5_test.grib',
 'ESACCI-LC-L4-LCCS-Map-300m-P1Y-2015-v2.0.7b.nc',
 'ESACCI-LC-L4-LCCS-Map-300m-P1Y-2015-v2.0.7b.nc.zip',
 'ESACCI-LC-Legend.csv',
 'GRUN_v1_GSWP3_WGS84_05_1902_2014.nc',
 'kenya_precip_JanJune.grib',
 'kenya_rainfall2.nc',
 'kenya_rainfall_.nc',
 'orography.grib',
 'pressure.grib']


# Download from API

In [11]:
from src.api_helpers import create_subset, Region, create_api_args, download_data

# initialise api client
c = cdsapi.Client()

# where do you want to save the data?
save_data_path = data_dir / "kenya_veg_v1.zip"

# get geog location & turn into a subset string
kenya_region = Region(
    name='kenya',
    lonmin=33.501,
    lonmax=42.283,
    latmin=-5.202,
    latmax=6.002,
)
kenya_subset = create_subset(kenya_region)

# get timesteps
years=np.arange(1980,2019)
months=np.arange(1,13)
times=[0]

# create the API request
# create_api_args
api_args = {
        'format':'zip',
        'variable':[
            'fapar','lai'
        ],
        'year':[
            '2014','2015','2016',
            '2017','2018'
        ],
        'month':[
            '01','02','03',
            '04','05','06',
            '07','08','09',
            '10','11','12'
        ],
        'day':[
            '03','13','21',
            '22','23','24'
        ],
        'version':'1.5.1',
        'area':kenya_subset,
    }

In [ ]:
# kenya_veg_path = download_data( 
#     cdsapi_client=c, 
#     cdstype='satellite-albedo', 
#     save_data_path=save_data_path, 
#     api_args=api_args
# )

In [15]:
out_paths = []
for variable in ['fapar','lai']:
    for year in np.arange(1999,2015):
        # build the api client request
        api_args = {
            'format':'zip',
            'variable':[
                variable
            ],
            'year':[
                f"year"
            ],
            'month':[
                '01','02','03',
                '04','05','06',
                '07','08','09',
                '10','11','12'
            ],
            'day':[
                '03','13','21',
                '22','23','24'
            ],
            'version':'1.4.1',
            'area':kenya_subset,
        },

        # where do you want to save the data?
        save_data_path = data_dir / f"kenya_veg_{variable}_{year}_v2.zip"

        path = download_data( 
            cdsapi_client=c, 
            cdstype='satellite-albedo', 
            save_data_path=save_data_path, 
            api_args=api_args
        )
        out_path.append(path)

2019-05-08 20:10:40,267 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/satellite-albedo
/Users/tommylees/miniconda3/envs/crop/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
2019-05-08 20:10:40,378 WARNING Recovering from HTTP error [500 ], attemps 0 of 500
2019-05-08 20:10:40,380 WARNING Retrying in 120 seconds
/Users/tommylees/miniconda3/envs/crop/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
2019-05-08 20:12:40,500 WARNING Recovering from HTTP error [500 ], attemps 1 of 500
2019-05-08 20:12:40,50

KeyboardInterrupt: 

In [16]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'satellite-soil-moisture',
    {
        'format':'zip',
        'variable':'volumetric_surface_soil_moisture',
        'type_of_sensor':'combined_passive_and_active',
        'time_aggregation':'month_average',
        'year':[
            '1978','1979','1980',
            '1981','1982','1983',
            '1984','1985','1986',
            '1987','1988','1989',
            '1990','1991','1992',
            '1993','1994','1995',
            '1996','1997','1998',
            '1999','2000','2001',
            '2002','2003','2004',
            '2005','2006','2007',
            '2008','2009','2010',
            '2011','2012','2013',
            '2014','2015','2016',
            '2017','2018'
        ],
        'month':[
            '01','02','03',
            '04','05','06',
            '07','08','09',
            '10','11','12'
        ],
        'day':'01',
        'type_of_record':'cdr',
        'version':'v201812.0.0'
    },
    'download.zip')

2019-05-08 20:15:40,446 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/satellite-soil-moisture
/Users/tommylees/miniconda3/envs/crop/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
2019-05-08 20:15:41,905 INFO Request is completed
2019-05-08 20:15:41,909 INFO Downloading http://136.156.132.235/cache-compute-0006/cache/data1/dataset-satellite-soil-moisture-fa7b947a-3329-47db-b3c5-581b1e164d08.zip to download.zip (319.1M)


KeyboardInterrupt: 